<a href="https://colab.research.google.com/github/RachitrajeshParihar/Convo__management/blob/main/convo_management_groq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Conversation Management & Classification using Groq API
This notebook demonstrates two primary tasks using the Groq API (OpenAI-compatible interface):

1. Conversation Management (Truncation & Summarization):
Manage long chat histories by truncating them by turns, characters, or words.
Implement periodic summarization every k runs, replacing history with a summary.


2. Conversation Classification with JSON Schema & Function Calling:
Use structured extraction (OpenAI-compatible function-calling) to classify chats and extract user details into a strict JSON Schema.
Validate results using jsonschema.


##Constraints:
No frameworks (Streamlit, Flask, etc.) — only standard Python + requests or OpenAI-compatible SDK.
Demonstrations are copy-paste runnable in Colab (API keys placeholder).
Includes tests and clear printed outputs at each stage.

In [ ]:
# ---------- STEP 1: Install required packages and dependencies----------

!pip install openai requests jsonschema -q

In [ ]:
# ---------- STEP 2: Configure API & Import Libraries ----------
import os
from getpass import getpass

# ============= CONFIG =============
# Always use the official Groq OpenAI-compatible API base
GROQ_API_BASE = "https://api.groq.com/openai/v1"

# Use a valid Groq-hosted model
GROQ_MODEL = "moonshotai/kimi-k2-instruct"   # ✅ Default recommendation

# Ask for API key securely
if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass("Enter your Groq API key: ")

GROQ_API_KEY = os.environ["GROQ_API_KEY"]

print("[INFO] Configured API settings. Base:", GROQ_API_BASE, "Model:", GROQ_MODEL)

[INFO] Configured API settings. Base: https://api.groq.com/openai/v1 Model: moonshotai/kimi-k2-instruct


In [ ]:
# ---------- STEP 3: OpenAI Client (Groq-Compatible) ----------
from openai import OpenAI
import requests

# Configure OpenAI client with Groq base
client = OpenAI(
    api_key=os.environ["GROQ_API_KEY"],
    base_url=GROQ_API_BASE
)

# requests fallback
def groq_request_fallback(prompt, model=GROQ_MODEL):
    url = f"{GROQ_API_BASE}/chat/completions"
    headers = {
        "Authorization": f"Bearer {os.environ['GROQ_API_KEY']}",
        "Content-Type": "application/json"
    }
    data = {
        "model": model,
        "messages": [{"role": "user", "content": prompt}]
    }
    response = requests.post(url, headers=headers, json=data)
    return response.json()

print("[INFO] OpenAI/Groq client initialized using model:", GROQ_MODEL)

[INFO] OpenAI/Groq client initialized using model: moonshotai/kimi-k2-instruct


In [ ]:
# ---------- STEP 4: Conversation Management Implementation ----------
# Functions for Task 1
def truncate_by_turns(history, n):
    return history[-n:]

def truncate_by_chars(history, max_chars):
    truncated, total = [], 0
    for msg in reversed(history):
        ml = len(msg["content"])
        if total + ml <= max_chars:
            truncated.insert(0, msg)
            total += ml
        else:
            break
    return truncated

def truncate_by_words(history, max_words):
    truncated, total = [], 0
    for msg in reversed(history):
        wl = len(msg["content"].split())
        if total + wl <= max_words:
            truncated.insert(0, msg)
            total += wl
        else:
            break
    return truncated

def summarize_with_model(text, model, client_or_requests="client"):
    if client_or_requests == "client":
        resp = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": f"Summarize: {text}"}]
        )
        return resp.choices[0].message.content.strip()
    else:
        resp = groq_request_fallback(f"Summarize: {text}", model=model)
        return resp["choices"][0]["message"]["content"].strip()

def periodic_summarize(history, run_count, k, summarizer_fn):
    if run_count % k == 0:
        text = "\n".join([f"{m['role']}: {m['content']}" for m in history])
        summary = summarizer_fn(text)
        return [{"role": "system", "content": f"[Summary {run_count}] {summary}"}]
    return history

print("[INFO] Conversation management functions defined.")

[INFO] Conversation management functions defined.


In [ ]:
# ---------- STEP 5: Demo Truncation ----------
sample_history = [
    {"role": "user", "content": "Hello, my name is Alice."},
    {"role": "assistant", "content": "Hi Alice, how can I help you today?"},
    {"role": "user", "content": "I’d like to book a flight to Paris."},
    {"role": "assistant", "content": "What dates are you considering?"},
    {"role": "user", "content": "Next week, preferably Monday."},
    {"role": "assistant", "content": "Got it. One ticket from NYC to Paris next Monday."}
]

print("Last 2 turns:", truncate_by_turns(sample_history, 2))
print("\nLimit 200 chars:", truncate_by_chars(sample_history, 200))
print("\nLimit 50 words:", truncate_by_words(sample_history, 50))

assert len(truncate_by_turns(sample_history, 2)) == 2
print("[TEST] Truncate by turns passed")

Last 2 turns: [{'role': 'user', 'content': 'Next week, preferably Monday.'}, {'role': 'assistant', 'content': 'Got it. One ticket from NYC to Paris next Monday.'}]

Limit 200 chars: [{'role': 'assistant', 'content': 'Hi Alice, how can I help you today?'}, {'role': 'user', 'content': 'I’d like to book a flight to Paris.'}, {'role': 'assistant', 'content': 'What dates are you considering?'}, {'role': 'user', 'content': 'Next week, preferably Monday.'}, {'role': 'assistant', 'content': 'Got it. One ticket from NYC to Paris next Monday.'}]

Limit 50 words: [{'role': 'user', 'content': 'Hello, my name is Alice.'}, {'role': 'assistant', 'content': 'Hi Alice, how can I help you today?'}, {'role': 'user', 'content': 'I’d like to book a flight to Paris.'}, {'role': 'assistant', 'content': 'What dates are you considering?'}, {'role': 'user', 'content': 'Next week, preferably Monday.'}, {'role': 'assistant', 'content': 'Got it. One ticket from NYC to Paris next Monday.'}]
[TEST] Truncate by turns

In [ ]:
# ---------- STEP 6: Periodic Summarization Demo ----------
history = [
    {"role": "user", "content": "Hello"},
    {"role": "assistant", "content": "Hi!"},
    {"role": "user", "content": "Book a flight?"},
    {"role": "assistant", "content": "When?"},
]

for run in range(1, 7):
    history = periodic_summarize(history, run, 3,
                                 lambda t: f"(Simulated summary of {len(t)} chars)")
    print(f"\nRun {run} -> history:")
    for msg in history:
        print(msg)


Run 1 -> history:
{'role': 'user', 'content': 'Hello'}
{'role': 'assistant', 'content': 'Hi!'}
{'role': 'user', 'content': 'Book a flight?'}
{'role': 'assistant', 'content': 'When?'}

Run 2 -> history:
{'role': 'user', 'content': 'Hello'}
{'role': 'assistant', 'content': 'Hi!'}
{'role': 'user', 'content': 'Book a flight?'}
{'role': 'assistant', 'content': 'When?'}

Run 3 -> history:
{'role': 'system', 'content': '[Summary 3] (Simulated summary of 64 chars)'}

Run 4 -> history:
{'role': 'system', 'content': '[Summary 3] (Simulated summary of 64 chars)'}

Run 5 -> history:
{'role': 'system', 'content': '[Summary 3] (Simulated summary of 64 chars)'}

Run 6 -> history:
{'role': 'system', 'content': '[Summary 6] (Simulated summary of 51 chars)'}


In [ ]:
# ---------- STEP 7: Task 2 JSON Schema & Extraction ----------
import jsonschema, json

user_schema = {
    "type": "object",
    "properties": {
        "name": {"type": "string"},
        "email": {"type": "string", "format": "email"},
        "phone": {"type": "string", "pattern": "^[0-9+() -]{7,20}$"},
        "location": {"type": "string"},
        "age": {"type": "integer", "minimum": 0, "maximum": 130}
    },
    "required": ["name", "email", "phone", "location"],
    "additionalProperties": False
}

def validate_with_jsonschema(obj, schema=user_schema):
    try:
        jsonschema.validate(obj, schema)
        return True, ""
    except jsonschema.exceptions.ValidationError as e:
        return False, str(e)

def call_extraction_function(chat_text, schema, model=GROQ_MODEL, client_or_requests="client"):
    instr = f"Extract fields (name, email, phone, location, age) in JSON {json.dumps(schema)}. Chat: {chat_text}"
    if client_or_requests == "client":
        resp = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": instr}],
            response_format={"type": "json_object"}
        )
        raw = resp.choices[0].message.content
    else:
        resp = groq_request_fallback(instr, model=model)
        raw = resp["choices"][0]["message"]["content"]

    try:
        parsed = json.loads(raw)
        return parsed, None
    except Exception as e:
        return None, str(e)

print("[INFO] JSON schema + extraction functions loaded (using model:", GROQ_MODEL, ")")

[INFO] JSON schema + extraction functions loaded (using model: moonshotai/kimi-k2-instruct )


In [ ]:
# ---------- STEP 8: Demo Extraction ----------
sample_chats = [
    "Hello, I'm John Doe. My email is john@example.com, I live in Berlin, I'm 29 years old, call me at +49123456789.",
    "Hi, Sarah Smith here. Location: Toronto. My contact is sarah.smith@mail.com, phone 416-555-0123. Age 35.",
    "Name: Peter Johnson. Email: peter.j@example.org. Location: New York. Mobile: (212) 555-9087."
]

for i, chat in enumerate(sample_chats, 1):
    print(f"\n--- Chat {i} ---")
    parsed, err = call_extraction_function(chat, user_schema)
    if err:
        print("Error parsing:", err)
    else:
        print("Extracted:", parsed)
        valid, errmsg = validate_with_jsonschema(parsed)
        print("Validation:", "PASSED " if valid else f"FAILED  {errmsg}")


--- Chat 1 ---
Extracted: {'name': 'John Doe', 'email': 'john@example.com', 'phone': '+49123456789', 'location': 'Berlin', 'age': 29}
Validation: PASSED 

--- Chat 2 ---
Extracted: {'name': 'Sarah Smith', 'email': 'sarah.smith@mail.com', 'phone': '416-555-0123', 'location': 'Toronto', 'age': 35}
Validation: PASSED 

--- Chat 3 ---
Extracted: {'name': 'Peter Johnson', 'email': 'peter.j@example.org', 'phone': '(212) 555-9087', 'location': 'New York'}
Validation: PASSED 
